# ENEA Technical Interview

Data insights from crash stats

----

1. Load CSVs, explore the data a bit. Visualise.
2. Look for insights on accidents, location, severity, weather(?), duration, time?

----

Brainstorming:

* Due to time constraints, it is more efficient to: think of a specific problem, explore data, then solve it or improve it, rather then explore data ,find a problem, then solve it:
    * Locality based problems - busy intersections, roundabouts, tram lines e.g. Toorak Rd Area?
    * Global problems, but targeted demographic: pedestrian safety at night, morning work cyclists, motorbikes on wet roads, etc.

----

Decided to look at:

* **Obstacles + Conditions in Motorcycle Accidents. Hidden dangers and where can we find improvements?**

EDA to look into:

* **Cars+Heavier Vehicles vs Motorcycles, injury + fatalies normalized by riders (approximate)**
* Locations of motorcycle crashes, fatalities. Are they in city, suburbian, rural, mountainous?
* Statistics of crashes, do they involve pedestrians?
* Approximate cost

Calculations to make:

* Relative risk difference between various conditions group by severity

Machine Learning to implement:

* XGBoost on various features to predict severity. Look at feature importance, feed it a few "example scenarios",

Finally:
* Brainstorm solutions based on ML outcome.

----

Notes:

* Weather_df contains columns for conditions, then "order" these conditions occured in. For this exercise, ignore the latter. We will load this, then one-hot on the named category, compress by summing along index, then merging.


In [2]:
import os
import functools

import numpy as np
import pandas as pd
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

pd.options.display.width = 180
pd.options.display.max_colwidth = 120
pd.options.display.max_rows = 100
pd.options.display.max_columns = 30

sns.reset_defaults()
sns.set_theme(context='notebook', style='ticks', font_scale=1, rc={
    'axes.spines.right': False,
    'axes.spines.top': False,
    'figure.figsize': (4,3),
})

print('numpy version', np.__version__)
print('pandas version', pd.__version__)

numpy version 1.21.6
pandas version 1.1.5


In [48]:
#!unzip "2000 to 2005 ACCIDENT.zip" -d "2000 to 2005 ACCIDENT"
#!unzip "ACCIDENT.zip" -d "2006 to 2020 ACCIDENT"

In [3]:
class CFG:
    
    num_samples = 20000
    
    data_dir = 'ACCIDENT'
    acc_csv = f'{data_dir}/ACCIDENT.csv'
    weather_csv = f'{data_dir}/ATMOSPHERIC_COND.csv'
    person_csv = f'{data_dir}/PERSON.csv'
    


In [35]:
"""utilities"""

def check_na_type(df):
    """check df columns for na type, and frequency before using fillna."""
    for i in df.columns[df.isna().any()]:
        print(df[i].apply(type).value_counts())
    return

def num_encoder(arr_):
    """used to rename columns after one-hot. encodes number to label, which is then passed onto 
    df.rename
    """
    encoder = {}
    for idx, i in enumerate([f"cond_{i.split('_')[1]}" for i in arr_]):
        encoder[idx] = i
    return encoder

In [5]:
def preprocess_df(df):
    """clean df, populate as we go along"""
    df_ = df.fillna('')
    
    return df_

acc_df = pd.read_csv(CFG.acc_csv)[:CFG.num_samples]
weather_df = pd.read_csv(CFG.weather_csv)[:CFG.num_samples]
person_df = pd.read_csv(CFG.person_csv)[:CFG.num_samples]

acc_df = preprocess_df(acc_df).set_index(['ACCIDENT_NO'])
weather_df = preprocess_df(weather_df).set_index(['ACCIDENT_NO'])
person_df = preprocess_df(person_df).set_index(['ACCIDENT_NO'])

#check unique_id
print(len(weather_df.index.unique()))
print(len(acc_df.index.unique()))
print(len(person_df.index.unique()))

#cursory check that ACCIDENT_NO is ordered, we'll take person_df, which is the csv of all people involved
#in an accident minus 1, and merge with the other dfs by ACCIDENT NO.
person_df = person_df[person_df.index != person_df.index[-1]]

/Users/keven/v_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/keven/v_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


19626
20000
7986


In [108]:
merged_list = [acc_df, weather_df, person_df]

merged_df = functools.reduce(lambda l, r: pd.merge(l, r, left_index=True, right_index=True), merged_list)


In [50]:
encoder = OneHotEncoder()
test = weather_df.drop(['ATMOSPH_COND', 'ATMOSPH_COND_SEQ'], axis=1).reset_index()
test = pd.merge(test, pd.DataFrame(encoder.fit_transform(test[['Atmosph Cond Desc']]).toarray()), left_index=True, right_index=True)
test = test.drop(['Atmosph Cond Desc'], axis=1)

cond_enc = num_encoder(encoder.get_feature_names_out())
test = test.rename(columns=cond_enc)

test = test.groupby(['ACCIDENT_NO']).sum()

test

,cond_Clear,cond_Dust,cond_Fog,cond_Not known,cond_Raining,cond_Smoke,cond_Snowing,cond_Strong winds
ACCIDENT_NO,,,,,,,,
T20060000010,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20060000018,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20060000022,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20060000023,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20060000026,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
T20070020900,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20070020902,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
T20070020904,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


4.0

In [34]:



cond_enc

{0: 'cond_Clear',
 1: 'cond_Dust',
 2: 'cond_Fog',
 3: 'cond_Not known',
 4: 'cond_Raining',
 5: 'cond_Smoke',
 6: 'cond_Snowing',
 7: 'cond_Strong winds'}

In [27]:
[i for i in zip([1,2],[2,3])]

[(1, 2), (2, 3)]

In [125]:
person_df[person_df.index == 'T20060018293']

,PERSON_ID,VEHICLE_ID,SEX,AGE,Age Group,INJ_LEVEL,Inj Level Desc,SEATING_POSITION,HELMET_BELT_WORN,ROAD_USER_TYPE,Road User Type Desc,LICENCE_STATE,PEDEST_MOVEMENT,POSTCODE,TAKEN_HOSPITAL,EJECTED_CODE
ACCIDENT_NO,,,,,,,,,,,,,,,,
T20060018293,01,,M,22,22-25,3,Other injury,,8,1,Pedestrians,V,1,3805,N,0
T20060018293,02,A,F,25,22-25,4,Not injured,LF,1,3,Passengers,V,0,3131,,0
T20060018293,A,A,M,23,22-25,4,Not injured,D,1,2,Drivers,V,0,,,0


In [126]:
acc_df[acc_df.index == 'T20060018293']

,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,EDITION,PAGE,GRID_REFERENCE_X,GRID_REFERENCE_Y,LIGHT_CONDITION,Light Condition Desc,NODE_ID,NO_OF_VEHICLES,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
ACCIDENT_NO,,,,,,,,,,,,,,,,,,,,,,,,,,,
T20060018293,7/05/2006,04:15:00,2,Struck Pedestrian,1,Sunday,100,PED NEAR SIDE. PED HIT BY VEHICLE FROM THE RIGHT.,MEL,40,2L,J,9,3,Dark Street lights on,204422,1,3,0,1,0,2,1,5,Not at intersection,3,50


array(['Atmosph Cond Desc_Clear', 'Atmosph Cond Desc_Dust',
       'Atmosph Cond Desc_Fog', 'Atmosph Cond Desc_Not known',
       'Atmosph Cond Desc_Raining', 'Atmosph Cond Desc_Smoke',
       'Atmosph Cond Desc_Snowing', 'Atmosph Cond Desc_Strong winds'],
      dtype=object)

'1.0.2'

(8, 3)